In [ ]:
operator fun <T> MutableList<MutableList<T>>.get(index: Pair<Int, Int>): T = this[index.first][index.second]

operator fun <T> MutableList<MutableList<T>>.set(index: Pair<Int, Int>, value: T) {
    this[index.first][index.second] = value
}

val grid = File("sample.txt").readLines().map {
    it.map { it.toString() }.toMutableList()
}.toMutableList()

fun <T> MutableList<MutableList<T>>.rotate() {
    val n = this.size
    val m = this[0].size

    for (i in 0 until n) {
        for (j in i + 1 until m) {
            val temp = this[i][j]
            this[i][j] = this[j][i]
            this[j][i] = temp
        }
    }

    for (i in 0 until m) {
        var top = 0
        var bottom = n - 1
        while (top < bottom) {
            val temp = this[top][i]
            this[top][i] = this[bottom][i]
            this[bottom][i] = temp
            top++
            bottom--
        }
    }
}


fun <T> List<List<T>>.print() {
    forEach { row ->
        println(row.joinToString(" "))
    }
}


lateinit var startingIndex: Pair<Int, Int>

for (i in grid.indices) {
    for (j in grid[i].indices)
        if (grid[i][j] == "^") {
            startingIndex = i to j
        }
}

var currentIndex: Pair<Int, Int> = startingIndex
var exit = false
while (!exit) {
    grid[currentIndex] = "^"

    val next = grid.getOrNull(currentIndex.first - 1)?.getOrNull(currentIndex.second)


    if (next == null) {
        exit = true
        break
    }


    if (grid[currentIndex.first - 1][currentIndex.second] == "#") {
        grid.rotate()

        currentIndex = grid.size - currentIndex.second - 1 to currentIndex.first
    }

    currentIndex = currentIndex.first - 1 to currentIndex.second
}

grid.map { it.count { it == "^" } }.sum()

In [4]:
import java.io.File

val grid = File("day6.txt").readLines().map {
    it.map { it.toString() }.toMutableList()
}.toMutableList()

fun <T> List<List<T>>.print() {
    forEach { row ->
        println(row.joinToString(" "))
    }
}

enum class Direction(val vector: Pair<Int, Int>) {
    NORTH(-1 to 0),
    EAST(0 to 1),
    SOUTH(1 to 0),
    WEST(0 to -1);

    fun next() = values()[(ordinal + 1) % values().size]
}

val obstacles: Set<Pair<Int, Int>>
    get() = grid.flatMapIndexed { i, row ->
        row.mapIndexedNotNull { j, cell ->
            if (cell == "#" || cell == "O") i to j else null
        }
    }.toSet()

class Guard(val grid: MutableList<MutableList<String>>) {
    val directionMap = mutableMapOf<Pair<Pair<Int, Int>, Direction>, Boolean>()

    val initialPos: Pair<Int, Int>
        get() = grid.mapIndexedNotNull { i, row ->
            row.mapIndexedNotNull { j, character ->
                if (character == "^") i to j else null
            }.firstOrNull()
        }.first()

    var initialDir = Direction.NORTH

    fun travese(
        obstacles: Set<Pair<Int, Int>>,
        onNewPositionVisited: (Pair<Int, Int>, Direction) -> Unit
    ) {
        var pos = initialPos
        var dir = initialDir

        while (pos.first in grid.indices && pos.second in grid.first().indices) {
            onNewPositionVisited(pos, dir)

            val next = pos.first + dir.vector.first to pos.second + dir.vector.second
            if (next in obstacles) {
                dir = dir.next()
            } else {
                pos = next
            }
        }
    }
}

var looped = 0

for (i in grid.indices) {
    for (j in grid.indices) {
        val visited: MutableSet<Pair<Pair<Int, Int>, Direction>> = mutableSetOf()
        val currObstacles: MutableSet<Pair<Int, Int>> = mutableSetOf(*obstacles.toTypedArray())
        currObstacles.add(i to j)
        try {
            Guard(grid).travese(currObstacles) { pos, dir ->
                if (pos to dir in visited) {
                    looped++
                    throw RuntimeException()
                }
                visited += pos to dir
            }
        } catch (_: RuntimeException) {
            continue
        }
    }
}


looped

1888